# American Black-Scholes PDE Solver using Crank-Nicolson Method

This notebook extends our European option solver to handle **American options**, which can be exercised at any time before expiration. This introduces an **optimal stopping problem** that requires solving a **Linear Complementarity Problem (LCP)**.

## The American Option Problem

For American options, we have the constraint:
$$V(S,t) \geq g(S,t)$$

where $g(S,t)$ is the **payoff function**. The PDE becomes:

$$\max\left\{\frac{\partial V}{\partial t} + \mathcal{L}V, \; g(S,t) - V(S,t)\right\} = 0$$

This creates an **exercise boundary** $S^*(t)$ where early exercise is optimal.

## 1. Import Required Libraries

Import the necessary libraries and our American option solver.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Import our American option solver
from american_black_scholes_solver import (
    AmericanBlackScholesConfig,
    american_crank_nicolson_solver,
    plot_american_results,
    compare_american_european,
    payoff_function
)

# Optional seaborn for enhanced plotting
try:
    import seaborn as sns
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
except ImportError:
    sns = None
    print("Seaborn not available, using default matplotlib style")

print("Libraries imported successfully!")

## 2. Configure American Option Parameters

Set up parameters for an American put option. American puts are more interesting than calls because they have significant early exercise premiums.

In [ ]:
# American Put Option Configuration
config = AmericanBlackScholesConfig(
    S_max=150.0,        # Maximum stock price for grid
    K=100.0,            # Strike price
    T=1.0,              # Time to expiration (1 year)
    r=0.05,             # Risk-free rate (5%)
    sigma=0.2,          # Volatility (20%)
    option_type='put',  # American put
    penalty_param=1e6   # Penalty parameter for constraint enforcement
)

print(f"American Option Parameters:")
print(f"Strike Price (K): ${config.K}")
print(f"Time to Maturity (T): {config.T} years")
print(f"Risk-free Rate (r): {config.r*100}%")
print(f"Volatility (σ): {config.sigma*100}%")
print(f"Option Type: {config.option_type}")
print(f"Penalty Parameter: {config.penalty_param:.0e}")

print(f"\nWhy American puts are interesting:")
print(f"- Can be exercised early when stock price falls")
print(f"- Exercise boundary depends on interest rates and volatility")
print(f"- Higher early exercise premium than calls")

## 3. Understanding the Penalty Method

The **penalty method** transforms the Linear Complementarity Problem into a nonlinear PDE:

$$\frac{\partial V}{\partial t} + \mathcal{L}V - \rho \max(g - V, 0) = 0$$

where $\rho$ is a large penalty parameter.

In [ ]:
# Demonstrate the penalty function
S_demo = np.linspace(50, 150, 100)
V_demo = 15 + 0.1 * S_demo  # Some arbitrary option value
payoff_demo = payoff_function(S_demo, config.K, config.option_type)

# Calculate penalty term
penalty_values = config.penalty_param * np.maximum(payoff_demo - V_demo, 0)

plt.figure(figsize=(15, 5))

# Plot 1: Option value vs payoff
plt.subplot(1, 3, 1)
plt.plot(S_demo, V_demo, 'b-', linewidth=2, label='Option Value')
plt.plot(S_demo, payoff_demo, 'r--', linewidth=2, label='Payoff')
plt.xlabel('Stock Price ($)')
plt.ylabel('Value ($)')
plt.title('Option Value vs Payoff')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Constraint violation
plt.subplot(1, 3, 2)
violation = np.maximum(payoff_demo - V_demo, 0)
plt.plot(S_demo, violation, 'g-', linewidth=2)
plt.xlabel('Stock Price ($)')
plt.ylabel('Constraint Violation')
plt.title('max(Payoff - Value, 0)')
plt.grid(True, alpha=0.3)

# Plot 3: Penalty term
plt.subplot(1, 3, 3)
plt.semilogy(S_demo, penalty_values + 1, 'purple', linewidth=2)
plt.xlabel('Stock Price ($)')
plt.ylabel('Penalty Term (log scale)')
plt.title(f'Penalty Term (ρ = {config.penalty_param:.0e})')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The penalty method enforces V ≥ payoff by adding large penalties when violated.")

## 4. Solve the American Option PDE

Now let's solve the American option pricing problem using our Crank-Nicolson solver with penalty method.

In [ ]:
# Grid parameters
N_S = 100   # Number of stock price points
N_t = 500   # Number of time points

print("Solving American put option...")
print(f"Grid: {N_S} × {N_t} points")
print(f"Stock price range: $0 to ${config.S_max}")
print(f"Time range: 0 to {config.T} years")

# Solve the American option
S_grid, t_grid, V_american, exercise_boundary = american_crank_nicolson_solver(
    config, N_S=N_S, N_t=N_t, method='penalty'
)

print(f"\nSolution completed!")
print(f"Solution shape: {V_american.shape}")
print(f"Exercise boundary range: ${exercise_boundary.min():.2f} to ${exercise_boundary.max():.2f}")

# Display some key values
atm_idx = np.argmin(np.abs(S_grid - config.K))
american_price = V_american[atm_idx, 0]
payoff_value = payoff_function(np.array([config.K]), config.K, config.option_type)[0]
time_value = american_price - payoff_value

print(f"\nAt-the-money results (S = ${config.K}):")
print(f"American option price: ${american_price:.6f}")
print(f"Intrinsic value: ${payoff_value:.6f}")
print(f"Time value: ${time_value:.6f}")

## 5. Visualize the American Option Solution

Plot comprehensive results including the exercise boundary.

In [ ]:
# Use our comprehensive plotting function
plot_american_results(S_grid, t_grid, V_american, exercise_boundary, config)

## 6. Compare American vs European Options

The key insight is that American options are **always worth at least as much** as their European counterparts, with the difference being the **early exercise premium**.

In [ ]:
# Compare American and European options
S_grid_comp, t_grid_comp, V_american_comp, V_european, exercise_boundary_comp = compare_american_european(
    config, N_S=N_S, N_t=N_t//2  # Use fewer time steps for comparison
)

## 7. Analyze the Exercise Boundary

The **exercise boundary** $S^*(t)$ is the critical stock price below which immediate exercise is optimal.

In [ ]:
# Detailed analysis of the exercise boundary
plt.figure(figsize=(15, 10))

# 1. Exercise boundary evolution
plt.subplot(2, 3, 1)
plt.plot(t_grid, exercise_boundary, 'r-', linewidth=3)
plt.axhline(y=config.K, color='k', linestyle='--', alpha=0.5, label='Strike Price')
plt.xlabel('Time (years)')
plt.ylabel('Exercise Boundary ($)')
plt.title('Optimal Exercise Boundary')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Exercise boundary as fraction of strike
plt.subplot(2, 3, 2)
boundary_ratio = exercise_boundary / config.K
plt.plot(t_grid, boundary_ratio, 'b-', linewidth=2)
plt.axhline(y=1.0, color='k', linestyle='--', alpha=0.5)
plt.xlabel('Time (years)')
plt.ylabel('S*/K')
plt.title('Exercise Boundary / Strike')
plt.grid(True, alpha=0.3)

# 3. Exercise decision map
plt.subplot(2, 3, 3)
S_mesh, T_mesh = np.meshgrid(S_grid, t_grid)
payoff_mesh = payoff_function(S_mesh, config.K, config.option_type)
exercise_decision = (V_american.T - payoff_mesh) < 1e-4  # Where V ≈ payoff

plt.contourf(S_mesh, T_mesh, exercise_decision.astype(int), levels=[0, 0.5, 1], 
             colors=['lightblue', 'lightcoral'], alpha=0.7)
plt.plot(exercise_boundary, t_grid, 'k-', linewidth=3, label='Exercise Boundary')
plt.xlabel('Stock Price ($)')
plt.ylabel('Time (years)')
plt.title('Exercise vs Hold Regions')
plt.legend()
plt.colorbar(label='Exercise (1) vs Hold (0)')

# 4. Early exercise premium along boundary
plt.subplot(2, 3, 4)
premium_boundary = []
for i, t in enumerate(t_grid[::10]):  # Sample every 10th point
    if i*10 < len(t_grid):
        S_boundary = exercise_boundary[i*10]
        S_idx = np.argmin(np.abs(S_grid - S_boundary))
        
        american_val = V_american[S_idx, i*10]
        payoff_val = payoff_function(np.array([S_boundary]), config.K, config.option_type)[0]
        premium_boundary.append(american_val - payoff_val)

plt.plot(t_grid[::10][:len(premium_boundary)], premium_boundary, 'g-', linewidth=2)
plt.xlabel('Time (years)')
plt.ylabel('Time Value at Boundary ($)')
plt.title('Time Value at Exercise Boundary')
plt.grid(True, alpha=0.3)

# 5. Sensitivity to interest rate (theoretical)
plt.subplot(2, 3, 5)
rates = np.array([0.01, 0.03, 0.05, 0.07, 0.10])
boundary_at_half_life = []

for r_test in rates:
    # Approximate boundary using formula for American put
    # S* ≈ K * r / (r + δ) where δ is related to volatility
    delta = 0.5 * config.sigma**2
    approx_boundary = config.K * r_test / (r_test + delta)
    boundary_at_half_life.append(approx_boundary)

plt.plot(rates * 100, boundary_at_half_life, 'o-', linewidth=2)
plt.axvline(x=config.r * 100, color='r', linestyle='--', alpha=0.7, label='Current Rate')
plt.xlabel('Interest Rate (%)')
plt.ylabel('Approximate Exercise Boundary ($)')
plt.title('Boundary Sensitivity to Interest Rate')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Delta across exercise boundary
plt.subplot(2, 3, 6)
dS = S_grid[1] - S_grid[0]
delta_american = np.gradient(V_american[:, 0], dS)
plt.plot(S_grid, delta_american, 'b-', linewidth=2, label='American Delta')
plt.axvline(x=exercise_boundary[0], color='r', linestyle='--', alpha=0.7, label='Exercise Boundary')
plt.xlabel('Stock Price ($)')
plt.ylabel('Delta')
plt.title('Delta at t=0')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print analysis
print(f"Exercise Boundary Analysis:")
print(f"Initial boundary (t=0): ${exercise_boundary[0]:.2f}")
print(f"Final boundary (t=T): ${exercise_boundary[-1]:.2f}")
print(f"Boundary range: ${exercise_boundary.min():.2f} - ${exercise_boundary.max():.2f}")
print(f"Average boundary: ${exercise_boundary.mean():.2f}")
print(f"\nBoundary as % of strike:")
print(f"Initial: {exercise_boundary[0]/config.K*100:.1f}%")
print(f"Final: {exercise_boundary[-1]/config.K*100:.1f}%")

## 8. Parameter Sensitivity Analysis

Analyze how the American option value and exercise boundary respond to different parameters.

In [ ]:
# Parameter sensitivity analysis
def analyze_parameter_sensitivity(base_config, param_name, param_values, N_S=80, N_t=200):
    """Analyze sensitivity to a parameter"""
    option_values = []
    exercise_boundaries = []
    
    for param_val in param_values:
        # Create modified config
        test_config = AmericanBlackScholesConfig(
            S_max=base_config.S_max,
            K=base_config.K,
            T=base_config.T,
            r=base_config.r,
            sigma=base_config.sigma,
            option_type=base_config.option_type,
            penalty_param=base_config.penalty_param
        )
        
        # Modify the specific parameter
        setattr(test_config, param_name, param_val)
        
        # Solve
        S_test, t_test, V_test, boundary_test = american_crank_nicolson_solver(
            test_config, N_S=N_S, N_t=N_t, method='penalty'
        )
        
        # Store ATM option value and initial boundary
        atm_idx = np.argmin(np.abs(S_test - base_config.K))
        option_values.append(V_test[atm_idx, 0])
        exercise_boundaries.append(boundary_test[0])
    
    return option_values, exercise_boundaries

# Test different parameters
plt.figure(figsize=(15, 10))

# 1. Volatility sensitivity
plt.subplot(2, 3, 1)
vol_values = np.linspace(0.1, 0.4, 7)
vol_prices, vol_boundaries = analyze_parameter_sensitivity(config, 'sigma', vol_values)
plt.plot(vol_values * 100, vol_prices, 'bo-', linewidth=2)
plt.xlabel('Volatility (%)')
plt.ylabel('Option Value ($)')
plt.title('Volatility Sensitivity')
plt.grid(True, alpha=0.3)

plt.subplot(2, 3, 4)
plt.plot(vol_values * 100, vol_boundaries, 'ro-', linewidth=2)
plt.xlabel('Volatility (%)')
plt.ylabel('Exercise Boundary ($)')
plt.title('Exercise Boundary vs Volatility')
plt.grid(True, alpha=0.3)

# 2. Interest rate sensitivity
plt.subplot(2, 3, 2)
rate_values = np.linspace(0.01, 0.10, 7)
rate_prices, rate_boundaries = analyze_parameter_sensitivity(config, 'r', rate_values)
plt.plot(rate_values * 100, rate_prices, 'go-', linewidth=2)
plt.xlabel('Interest Rate (%)')
plt.ylabel('Option Value ($)')
plt.title('Interest Rate Sensitivity')
plt.grid(True, alpha=0.3)

plt.subplot(2, 3, 5)
plt.plot(rate_values * 100, rate_boundaries, 'ro-', linewidth=2)
plt.xlabel('Interest Rate (%)')
plt.ylabel('Exercise Boundary ($)')
plt.title('Exercise Boundary vs Interest Rate')
plt.grid(True, alpha=0.3)

# 3. Time to maturity sensitivity
plt.subplot(2, 3, 3)
time_values = np.linspace(0.25, 2.0, 7)
time_prices, time_boundaries = analyze_parameter_sensitivity(config, 'T', time_values)
plt.plot(time_values, time_prices, 'mo-', linewidth=2)
plt.xlabel('Time to Maturity (years)')
plt.ylabel('Option Value ($)')
plt.title('Time to Maturity Sensitivity')
plt.grid(True, alpha=0.3)

plt.subplot(2, 3, 6)
plt.plot(time_values, time_boundaries, 'ro-', linewidth=2)
plt.xlabel('Time to Maturity (years)')
plt.ylabel('Exercise Boundary ($)')
plt.title('Exercise Boundary vs Time to Maturity')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Parameter Sensitivity Summary:")
print("\n1. Volatility:")
print(f"   - Higher volatility → Higher option value")
print(f"   - Higher volatility → Lower exercise boundary (hold longer)")
print("\n2. Interest Rate:")
print(f"   - Higher interest rate → Lower put value")
print(f"   - Higher interest rate → Higher exercise boundary (exercise sooner)")
print("\n3. Time to Maturity:")
print(f"   - Longer time → Higher option value")
print(f"   - Longer time → Lower exercise boundary (more time value)")

## 9. American Call vs Put Analysis

Compare American calls and puts to understand when early exercise is valuable.

In [ ]:
# Compare American call and put
print("Comparing American Call vs Put...")

# American call configuration
call_config = AmericanBlackScholesConfig(
    S_max=config.S_max,
    K=config.K,
    T=config.T,
    r=config.r,
    sigma=config.sigma,
    option_type='call',
    penalty_param=config.penalty_param
)

# Solve American call
S_call, t_call, V_call, boundary_call = american_crank_nicolson_solver(
    call_config, N_S=N_S//2, N_t=N_t//2, method='penalty'
)

# Plot comparison
plt.figure(figsize=(15, 10))

# 1. Option values at t=0
plt.subplot(2, 3, 1)
plt.plot(S_grid, V_american[:, 0], 'r-', linewidth=2, label='American Put')
plt.plot(S_call, V_call[:, 0], 'b-', linewidth=2, label='American Call')
put_payoff = payoff_function(S_grid, config.K, 'put')
call_payoff = payoff_function(S_grid, config.K, 'call')
plt.plot(S_grid, put_payoff, 'r:', alpha=0.7, label='Put Payoff')
plt.plot(S_grid, call_payoff, 'b:', alpha=0.7, label='Call Payoff')
plt.xlabel('Stock Price ($)')
plt.ylabel('Option Value ($)')
plt.title('American Call vs Put Values')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Exercise boundaries
plt.subplot(2, 3, 2)
plt.plot(t_grid, exercise_boundary, 'r-', linewidth=2, label='Put Exercise Boundary')
plt.plot(t_call, boundary_call, 'b-', linewidth=2, label='Call Exercise Boundary')
plt.axhline(y=config.K, color='k', linestyle='--', alpha=0.5, label='Strike')
plt.xlabel('Time (years)')
plt.ylabel('Exercise Boundary ($)')
plt.title('Exercise Boundaries')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Early exercise premium comparison
plt.subplot(2, 3, 3)
# For this comparison, we'd need European solutions too
# Let's show time value instead
put_time_value = V_american[:, 0] - put_payoff
call_time_value = V_call[:, 0] - call_payoff
plt.plot(S_grid, put_time_value, 'r-', linewidth=2, label='Put Time Value')
plt.plot(S_call, call_time_value, 'b-', linewidth=2, label='Call Time Value')
plt.xlabel('Stock Price ($)')
plt.ylabel('Time Value ($)')
plt.title('Time Value Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Exercise decision maps
plt.subplot(2, 3, 4)
S_mesh_put, T_mesh_put = np.meshgrid(S_grid, t_grid)
payoff_mesh_put = payoff_function(S_mesh_put, config.K, 'put')
exercise_decision_put = (V_american.T - payoff_mesh_put) < 1e-4
plt.contourf(S_mesh_put, T_mesh_put, exercise_decision_put.astype(int), 
             levels=[0, 0.5, 1], colors=['lightblue', 'lightcoral'], alpha=0.7)
plt.plot(exercise_boundary, t_grid, 'k-', linewidth=2)
plt.xlabel('Stock Price ($)')
plt.ylabel('Time (years)')
plt.title('Put Exercise Region')

plt.subplot(2, 3, 5)
S_mesh_call, T_mesh_call = np.meshgrid(S_call, t_call)
payoff_mesh_call = payoff_function(S_mesh_call, config.K, 'call')
exercise_decision_call = (V_call.T - payoff_mesh_call) < 1e-4
plt.contourf(S_mesh_call, T_mesh_call, exercise_decision_call.astype(int), 
             levels=[0, 0.5, 1], colors=['lightblue', 'lightgreen'], alpha=0.7)
plt.plot(boundary_call, t_call, 'k-', linewidth=2)
plt.xlabel('Stock Price ($)')
plt.ylabel('Time (years)')
plt.title('Call Exercise Region')

# 6. Put-Call relationship
plt.subplot(2, 3, 6)
# Put-call parity relationship for American options
# American Put + Stock ≥ American Call + PV(Strike)
pv_strike = config.K * np.exp(-config.r * config.T)
put_call_diff = V_american[:, 0] + S_grid - V_call[:, 0] - pv_strike
plt.plot(S_grid, put_call_diff, 'g-', linewidth=2)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
plt.xlabel('Stock Price ($)')
plt.ylabel('Put + S - Call - PV(K) ($)')
plt.title('American Put-Call Relationship')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("\n1. American Puts:")
print(f"   - Exercise boundary: Below ${exercise_boundary[0]:.2f} at t=0")
print(f"   - Early exercise when stock price falls significantly")
print(f"   - Higher early exercise premium than calls")

print("\n2. American Calls:")
print(f"   - Exercise boundary: Above ${boundary_call[0]:.2f} at t=0")
print(f"   - Early exercise typically only near expiration")
print(f"   - Lower early exercise premium (often ≈ European)")

print("\n3. When to Exercise Early:")
print(f"   - Puts: When time value < dividend savings (if any)")
print(f"   - Calls: When time value < dividend yield × stock price")
print(f"   - Both: When option is deep in-the-money")

## Summary

This notebook has successfully demonstrated American option pricing using the Crank-Nicolson method with constraint handling. Key achievements:

### Technical Implementation:
1. **Penalty Method**: Transforms the LCP into a nonlinear PDE
2. **Exercise Boundary**: Automatically computed during solution
3. **Constraint Enforcement**: Ensures $V \geq g$ at all points
4. **Numerical Stability**: Maintains Crank-Nicolson stability properties

### Financial Insights:
1. **Early Exercise Premium**: American options always worth ≥ European
2. **Exercise Boundaries**: Depend on volatility, interest rates, and time
3. **Put vs Call**: American puts have higher early exercise value
4. **Parameter Sensitivity**: Exercise decisions respond to market conditions

### Applications:
- **Risk Management**: Better hedging with early exercise features
- **Trading**: Optimal exercise timing strategies
- **Pricing**: More accurate valuation for American-style derivatives
- **Research**: Framework for exotic American options

The solver provides a robust foundation for pricing any American-style derivative and can be extended to handle dividends, stochastic volatility, and other complexities.